# Physics 129: Final Project #4 

# Measuring Matter Antimatter Asymmetries at the Large Hadron Collider

![](http://lhcb-public.web.cern.ch/lhcb-public/en/LHCb-outreach/multimedia/LHCbDetectorpnglight1.png)

### Learning objectives
In this project you will:

- Use data collected by the LHCb detector to study Direct CP violation in the decay $B^{\pm} \rightarrow K^{\pm} K^+ K^-$
- Characterize the resonant structure of the 3-body decay using Dalitz plots
- Extend the study of this decay mode in new directions

The LHCb experiment studies the properties of bottom and charm hadrons.  One of the main goals of the exoeriment is to measure CP violating effects in multiple channels and to compare these effects to the predictions of the Standard Model.  The data provided here are a subset of that released by LHCb as part of the CERN Open Data project.  You can find the complete data samples using the link provided in https://opendata.cern.ch/record/4902  If you are familiar with the Root data analysis package and would like to use Root to perform your analysis, you are free to download the data directly.

In this project, you will reproduce the measurement of the direct CP violation in the decay $B^{\pm} \rightarrow K^{\pm} K^+K^-$ that LHCb published in https://arxiv.org/pdf/1306.1246.pdf 

### Part 1. Importing and Analyzing the Raw Data

The following code reads the LHCb data.  The data have been pre-selected to require that there be exactly 3 charged tracks each of which has a likelihood of being a kaon of at leat 50% and none of which is consistent with being a muon.

In [5]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# There are two input files: One contains data with the magnetic field pointing up and the other with the 
# magnet pointing down
files = ["B2KKK_MagnetUp.csv","B2KKK_MagnetDown.csv"]

# Put the data fore each event into arrays.  Here 1, 2 and 3 refer to the first, second and third particle
B_FlightDistance = []
B_VertexChi2 = []
H1_PX = []
H1_PY = []
H1_PZ = []
H1_ProbK = []
H1_ProbPi = []
H1_Charge = []
H1_IPChi2 = []
H2_PX = []
H2_PY = []
H2_PZ = []
H2_ProbK = []
H2_ProbPi = []
H2_Charge = []
H2_IPChi2 = []
H3_PX = []
H3_PY = []
H3_PZ = []
H3_ProbK = []
H3_ProbPi = []
H3_Charge = []
H3_IPChi2 = []


for file in files:
  for line in open(file,"r"):
    line = line.strip()
 ##   if "#" in line :
 ##       Uncomment this to print the information provided at the beginning of the data files       
 ##       print(line)
    if(not "#" in line):
        info = line.split(",")
        B_FlightDistance.append(float(info[0]))
        B_VertexChi2.append(float(info[1]))
        H1_PX.append(float(info[2]))
        H1_PY.append(float(info[3]))
        H1_PZ.append(float(info[4]))
        H1_ProbK.append(float(info[5]))
        H1_ProbPi.append(float(info[6]))
        H1_Charge.append(int(info[7]))
        H1_IPChi2.append(float(info[8]))
        H2_PX.append(float(info[9]))
        H2_PY.append(float(info[10]))
        H2_PZ.append(float(info[11]))
        H2_ProbK.append(float(info[12]))
        H2_ProbPi.append(float(info[13]))
        H2_Charge.append(int(info[14]))
        H2_IPChi2.append(float(info[15]))
        H3_PX.append(float(info[16]))
        H3_PY.append(float(info[17]))
        H3_PZ.append(float(info[18]))
        H3_ProbK.append(float(info[19]))
        H3_ProbPi.append(float(info[20]))
        H3_Charge.append(int(info[21]))
        H3_IPChi2.append(float(info[22]))


print("Number of events read is ",len(H1_PX))


Number of events read is  26959


These data contain both signal and background events.  By plotting the invariant mass obtained by combining the 3 tracks under the assumption that all the particles are kaons you can identify a signal region near the $B^\pm$ mass and background regions at higher and lower mass.   Make separate histograms for the case where the charge of the candidate is positive and when it is negative. Your histograms should look similar to Figure 1b of the LHCb paper. 

###  Part II: Searching for global matter anti-matter differences

In this section you will start to study matter antimatter differences (CP Violation). Here 'global' means that you are looking for differences across all ranges of energy and momentum (the kinematics) of the kaons into which the charge B mesons have decayed. Later we look at 'local' differences in different kinematic regions.

Select candidate $B^{\pm} \rightarrow K^{\pm}K^+K^-$ with invariant mass close to that of the $B^{\pm}$ and calculate the matter-antimatter asymmetry 

$$
A_{global} = \frac{N_{-}-N_{+}}{N_{-}+N_{+}}
$$

where $N_{-}$ ($N_{+}$) is the number of observed decays of $B^-$ ($B^+$) to decay to 3 kaons.

### Part III: Dalitz plots and two body resonances

In this stage we introduce you to an important technique for analysing decays of one particle (your charged B meson) into three bodies (the three kaons). This is known as a Dalitz plot. 

The decay of the B meson can proceed either directly to the three-body final state or via an intermediate particle. For example, B<sup>+</sup> → K<sup>+</sup>K<sup>+</sup>K<sup>−</sup>, could proceed through the decay  B<sup>+</sup> → K<sup>+</sup>R<sup>0</sup>, where R<sup>0</sup> is a neutral particle resonance which can decay R<sup>0</sup> → K<sup>+</sup>K<sup>-</sup>. Dalitz plots can be used to identify these resonances which are visible as bands on the Dalitz plot.

More information about these plots and why these are used in particle physics research can be found in [Dalitz Plot Introduction](DalitzPlots.ipynb).  This introduction was prepared by LHCb for the open data project.

The kinematics of a three-body decay can be fully described using only two variables. The energies and momenta of the three kaons are not independent of each other as they all come from the decay of a B meson and energy and momentum are conserved. The axes of the plots conventionally are the squared invariant masses of two pairs of the decay products. It is a 2D plot, the x and y axes are both squared masses and the density of points in the plot shows the structure.

Consider our decay B<sup>+</sup> → K<sup>+</sup><sub>1</sub>K<sup>+</sup><sub>2</sub>K<sup>−</sup><sub>3</sub>, where we have numbered the kaons 1,2,3 to distinguish them. We can calculate the invariant mass of three possible combinations that could correspond to intermediate resonances R<sup>++</sup><sub>1</sub> → K<sup>+</sup><sub>1</sub>K<sup>+</sup><sub>2</sub>, R<sup>0</sup><sub>2</sub> → K<sup>+</sup><sub>1</sub>K<sup>-</sup><sub>3</sub>, and R<sup>0</sup><sub>3</sub> → K<sup>+</sup><sub>3</sub>K<sup>-</sup><sub>3</sub>. 

The potential R<sup>++</sup><sub>1</sub> would be a doubly charged resonance. We would not expect to see any resonances corresponding to this as mesons are composed of one quark and one anti-quark and their charges cannot add up to two units.

The potential R<sup>0</sup><sub>2</sub> and R<sup>0</sup><sub>3</sub> correspond to configurations in which we could see resonances. Hence you should compute the invariant mass combinations for these. The square of these masses should be used as the Dalitz variables.  You can make a further improvement to allow you to observe the resonances easier. Your resonances R<sup>0</sup><sub>2</sub> and R<sup>0</sup><sub>3</sub> are both composed of the same particle types, K<sup>+</sup>K<sup>-</sup>, and hence have the same distributions. It is useful to impose an ordering which distinguishes the resonances. We can call the resonances R<sup>0</sup><sub>Low</sub> and R<sup>0</sup><sub>High</sub>. In each event R<sup>0</sup><sub>Low</sub> is the resonance with the lower mass and the other corresponds to the higher mass combination of kaons. You can now use the mass of these ordered resonances as your Dalitz plot variables, thus effectively "folding" your Dalitz plot so that one axis always has a higher value than the other.


In [7]:
# Make a Dalitz Plot

You can now use your Dalitz plot to identify the intermediate resonances that you see in your plots. The resonances will have shown up as bands of higher density of points on the plots. You can use the [particle data group](http://pdg.lbl.gov/2015/tables/contents_tables.html) tables of mesons to identify which particles these correspond to. The tables give the masses and widths of the particles and their decay modes. You are looking for mesons with the masses corresponding to where you see the bands and that decay into K<sup>+</sup>K<sup>-</sup>.

### Part IV:  Searching for  matter anti-matter differences in charmless $B^\pm$ decays

The analysis performed here is to study the CP violation in the charmless B meson decays to kaons. "charmless" means that the decay does not proceed through a charm quark. However, the most frequent decay of the B mesons occur through the *b* quark decaying into a *c* quark. The majority of these events can be removed by rejecting the events that are proceeding through a D<sup>0</sup> meson (which contains the charm quark).

Using the variables from your Dalitz plot, remove the $D^0$ decays and measure the CP asymmetry for charmless $B^\pm$ decays. 


In [ ]:
# Your analysis goes here

### Extensions

Here are some ideas of how you could explore the data further, but you are welcome to take your project in a different direction.

* **CP Asymmetries in the with and without resonances**
For the case of charmless $B^\pm$ decays, compare the asymmetry you observe for events where two of the kaons are the decay products if a $\phi$ to the "non-responant" case where the three kaons are not produced through an intermediate resonance 

* **Systematic Uncertainties**
In this analysis you considered the statistical uncertainty on the result. This occurs as a result of having only a limited number of events. In addition there are [systematic uncertainties](https://en.wikipedia.org/wiki/Observational_error#Systematic_versus_random_error), these arise from biases in your measurement. One source of uncertainty is a possible asymmetry in the efficiency for reconstructing positively and negatively charged tracks. In this case, the LHCb detector could be more efficient for detecting either the $B^{+}$  or the $B^{-}$ final state. This difference arises because the positive and negative kaons will be bent by the magnet indifferent directions in the detector. If the efficiency of the detector is higher in one region than another this will to a charge asymmetry in the detection efficiency. For this reason the magnetic field of the LHCb detector is regularly reversed. You used data in this analysis in which the magnetic field was both up and down and hence the effect will (partially) cancel. By comparing results for the two magnet polarities separately you can check the size of this effect. When loading the data above both polarities were combined, you can instead load them independently to measure the difference between the two datasets. estimate the number of background events under the mass peak.

* **Fitting distributions** The next level of sophistication in the analysis requires fitting the distributions of events that are observed in the B mass distribution in order to estimate the yield of signal events and background events. You can see how this is done in the LHCb paper on the analysis. 

* **Measuring the CP Asymmetry for $B\rightarrow D^{0}K$ event** The published LHCb analysis removes events where two of the kaons are decay produces of a $D^0$.  You could study the asymmetry separately for such events.

* **Measurement of the lifetime of the $B^0$ meson** The data above include the flight length for each event.  You can therefore measure the $B^0$ lifetime.  Note:  To remove background, the data pre-selection requred a minimum value for the flight length.  You can determine that value from the data.

